# Part Combination Space Exploration

This is a notebook developed to leverage the new SysML v2 semantics for nested features and instantiation of models to generate instances of M1 system models as feedstock for analysis pipelines.

In [4]:
from __future__ import print_function

import requests
import getpass
import math

import time
import sysml_v2_api_client
from sysml_v2_api_client.rest import ApiException
from pprint import pprint

import json
import networkx as NX
import matplotlib.pyplot as plt

import random
import copy

import pymbe.api as pm
#from pymbe.model_loading import ModelingSession as Session
#from pymbe.interpretation.interpretation import RandomGenerationStrategy as RGS

ImportError: cannot import name 'NotFoundError' from 'ipyelk.exceptions' (/opt/tljh/user/envs/pyMBE/lib/python3.9/site-packages/ipyelk/exceptions.py)

In [2]:
import pymbe.query as pmQuery

# Configure API Server Connection

In [3]:
sysml_api_base_url = 'http://sysml2.intercax.com:9000' # http://sysml2-sst.intercax.com:9000

## Activate APIs

Connect the API classes to the server

In [4]:
configuration = sysml_v2_api_client.Configuration(
    host = sysml_api_base_url
)

projects_api_instance = None

with sysml_v2_api_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    project_api_instance = sysml_v2_api_client.ProjectApi(api_client)
    
commits_api_instance = None

with sysml_v2_api_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    commits_api_instance = sysml_v2_api_client.CommitApi(api_client)
    
elements_api_instance = None

with sysml_v2_api_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    elements_api_instance = sysml_v2_api_client.ElementApi(api_client)

## Pull down commits and elements catalogs

With the API handles, use the pre-built methods to get lists of commits and elements.

In [5]:
project_api_instance

Get a list of my active projects

In [6]:
my_projects = [my_proj for my_proj in project_api_instance.get_projects() if my_proj.name.find('jdehart') > -1]

In [7]:
# add project to dropdown
model_names = []
for key, value in enumerate(my_projects):
    model_names.append(my_projects[key].name)
model_names

['jdehart_sdp Sun Jun 06 21:33:29 UTC 2021',
 'jdehart_sdp_A Sun Jun 06 22:59:53 UTC 2021',
 'jdehart_ModelExample_A Fri Jun 04 11:12:17 UTC 2021',
 'jdehart_sdp_A Tue Jun 08 00:29:22 UTC 2021',
 'jdehart_demo Tue Jun 22 16:24:27 UTC 2021',
 'jdehart_partsModel Sun Jun 06 21:14:19 UTC 2021']

In [8]:
from IPython.display import display
from ipywidgets import Dropdown

def dropdown_eventhandler(change):
    global model_id
    model_id = change.new

option_list = model_names
dropdown = Dropdown(description="Select Project:", options=option_list)
dropdown.observe(dropdown_eventhandler, names='index')
display(dropdown)

Dropdown(description='Select Project:', options=('jdehart_sdp Sun Jun 06 21:33:29 UTC 2021', 'jdehart_sdp_A Su…

In [13]:
try:
    # Get commits by project
    commits_response = commits_api_instance.get_commits_by_project(my_projects[model_id].id)
    pprint(commits_response)
except ApiException as e:
    print("Exception when calling CommitApi->get_commits_by_project: %s\n" % e)

[{'change': None,
 'id': 'b226c0e8-f977-4d8f-93b4-0c8ee53e7c10',
 'owning_project': {'id': 'd700ba65-82c3-4927-9d15-1cf02122bd1d'},
 'previous_commit': None,
 'type': 'Commit'}]


In [14]:
elements = []

try:
    # Get commits by project
    elements = elements_api_instance.get_elements_by_project_commit(my_projects[model_id].id, commits_response[0].id)
except ApiException as e:
    print("Exception when calling ElementApi->get_elements_by_project_commit: %s\n" % e)

In [15]:
len(elements)

14

In [16]:
pprint(elements)

[{'id': '0d1dfeea-5169-47ab-91ae-1bc9efdfdc94',
 'identifier': '0d1dfeea-5169-47ab-91ae-1bc9efdfdc94',
 'type': 'AttributeUsage'},
 {'id': '16f048dd-8d36-4e42-9ffc-811fca8114c9',
 'identifier': '16f048dd-8d36-4e42-9ffc-811fca8114c9',
 'type': 'PartUsage'},
 {'id': '433fa4af-5e9b-4c64-bf00-f37cba8cfaac',
 'identifier': '433fa4af-5e9b-4c64-bf00-f37cba8cfaac',
 'type': 'Import'},
 {'id': '4635757b-6acb-41e1-abef-17c34a43c091',
 'identifier': '4635757b-6acb-41e1-abef-17c34a43c091',
 'type': 'DataType'},
 {'id': '606cd553-dfe1-4606-9656-df9fb7cbe730',
 'identifier': '606cd553-dfe1-4606-9656-df9fb7cbe730',
 'type': 'FeatureMembership'},
 {'id': '76445bf5-19bb-468c-986d-3d065552c05c',
 'identifier': '76445bf5-19bb-468c-986d-3d065552c05c',
 'type': 'Package'},
 {'id': '8b138df6-ce13-47f5-a139-9db5324ad92c',
 'identifier': '8b138df6-ce13-47f5-a139-9db5324ad92c',
 'type': 'FeatureMembership'},
 {'id': 'a8eac056-14b5-4e33-9941-f707b0d2b2c5',
 'identifier': 'a8eac056-14b5-4e33-9941-f707b0d2b2c5',


# Gather Element Data

Since the generated API doesn't have much detail for elements, need to do this more hands-on.

Not elegant below to just have a larger page size, should implement paging later on.

In [17]:
elements_url = (sysml_api_base_url +
                '/projects/{0}/commits/{1}/elements?page[size]=2000').format(my_projects[model_id].id, commits_response[0].id)

In [18]:
elements_url

'http://sysml2.intercax.com:9000/projects/d700ba65-82c3-4927-9d15-1cf02122bd1d/commits/b226c0e8-f977-4d8f-93b4-0c8ee53e7c10/elements?page[size]=2000'

In [19]:
elements_response = requests.get(
    elements_url
)

In [16]:
elements_data = elements_response.json()

Split the elements into relationships and non-relationships. This will let us work with graph representations and a graph understanding of the underlying model.

In [17]:
len(elements_data)

14

In [18]:
from IPython.display import JSON
JSON(elements_data)

<IPython.core.display.JSON object>

In [19]:
type(elements_data)

list

In [22]:
import pandas as pd
df = pd.DataFrame(elements_data)
df

,@type,@id,aliasId,attributeDefinition,documentation,documentationComment,effectiveName,endFeature,endOwningType,feature,...,memberName,membershipOwningNamespace,ownedMemberElement,ownedMemberFeature,owningFeatureOfType,filterCondition,general,owningFeature,specific,typedFeature
0,AttributeUsage,0d1dfeea-5169-47ab-91ae-1bc9efdfdc94,[],[{'@id': '4635757b-6acb-41e1-abef-17c34a43c091'}],[],[],None,[],NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PartUsage,16f048dd-8d36-4e42-9ffc-811fca8114c9,[],NaN,[],[],None,[],NaN,[{'@id': 'da4ba517-7f32-4652-890e-1200ea637f7b'}],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Import,433fa4af-5e9b-4c64-bf00-f37cba8cfaac,[],NaN,[],[],None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DataType,4635757b-6acb-41e1-abef-17c34a43c091,[],NaN,[],[],None,[],NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FeatureMembership,606cd553-dfe1-4606-9656-df9fb7cbe730,[],NaN,[],[],None,NaN,NaN,NaN,...,mass,{'@id': '16f048dd-8d36-4e42-9ffc-811fca8114c9'},{'@id': 'da4ba517-7f32-4652-890e-1200ea637f7b'},{'@id': 'da4ba517-7f32-4652-890e-1200ea637f7b'},NaN,NaN,NaN,NaN,NaN,NaN
5,Package,76445bf5-19bb-468c-986d-3d065552c05c,[],NaN,[],[],None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
6,FeatureMembership,8b138df6-ce13-47f5-a139-9db5324ad92c,[],NaN,[],[],None,NaN,NaN,NaN,...,mass,{'@id': 'a8eac056-14b5-4e33-9941-f707b0d2b2c5'},{'@id': '0d1dfeea-5169-47ab-91ae-1bc9efdfdc94'},{'@id': '0d1dfeea-5169-47ab-91ae-1bc9efdfdc94'},NaN,NaN,NaN,NaN,NaN,NaN
7,PartUsage,a8eac056-14b5-4e33-9941-f707b0d2b2c5,[],NaN,[],[],None,[],NaN,[{'@id': '0d1dfeea-5169-47ab-91ae-1bc9efdfdc94'}],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,FeatureTyping,ab152282-9c4d-4fde-905d-e3b7addfbe75,[],NaN,[],[],None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,{'@id': '4635757b-6acb-41e1-abef-17c34a43c091'},{'@id': '0d1dfeea-5169-47ab-91ae-1bc9efdfdc94'},{'@id': '0d1dfeea-5169-47ab-91ae-1bc9efdfdc94'},{'@id': '0d1dfeea-5169-47ab-91ae-1bc9efdfdc94'}
9,FeatureTyping,ac4a65ea-eaf7-4b5b-84a7-dcc5c9042abf,[],NaN,[],[],None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,{'@id': '4635757b-6acb-41e1-abef-17c34a43c091'},{'@id': 'da4ba517-7f32-4652-890e-1200ea637f7b'},{'@id': 'da4ba517-7f32-4652-890e-1200ea637f7b'},{'@id': 'da4ba517-7f32-4652-890e-1200ea637f7b'}


In [32]:
df[["@id","@type","name","owningFeatureMembership","owningRelatedElement"]]

,@id,@type,name,owningFeatureMembership,owningRelatedElement
0,0d1dfeea-5169-47ab-91ae-1bc9efdfdc94,AttributeUsage,mass,{'@id': '8b138df6-ce13-47f5-a139-9db5324ad92c'},NaN
1,16f048dd-8d36-4e42-9ffc-811fca8114c9,PartUsage,A,None,NaN
2,433fa4af-5e9b-4c64-bf00-f37cba8cfaac,Import,None,NaN,{'@id': '76445bf5-19bb-468c-986d-3d065552c05c'}
3,4635757b-6acb-41e1-abef-17c34a43c091,DataType,Real,NaN,NaN
4,606cd553-dfe1-4606-9656-df9fb7cbe730,FeatureMembership,None,NaN,{'@id': '16f048dd-8d36-4e42-9ffc-811fca8114c9'}
5,76445bf5-19bb-468c-986d-3d065552c05c,Package,jdehart_demo,NaN,NaN
6,8b138df6-ce13-47f5-a139-9db5324ad92c,FeatureMembership,None,NaN,{'@id': 'a8eac056-14b5-4e33-9941-f707b0d2b2c5'}
7,a8eac056-14b5-4e33-9941-f707b0d2b2c5,PartUsage,B,None,NaN
8,ab152282-9c4d-4fde-905d-e3b7addfbe75,FeatureTyping,None,NaN,{'@id': '0d1dfeea-5169-47ab-91ae-1bc9efdfdc94'}
9,ac4a65ea-eaf7-4b5b-84a7-dcc5c9042abf,FeatureTyping,None,NaN,{'@id': 'da4ba517-7f32-4652-890e-1200ea637f7b'}
